In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import pandas as pd
from nltk import everygrams, word_tokenize
import numpy as np

In [2]:
df = pd.read_csv('100 MCRs - 0407 - Sheet1.csv')

In [3]:
s = df['Summary'][4]
s = [i.lower() for i in s.split('\n') if i!='']

In [4]:
# s = [i.lower() for i in s.split('\n') if i!='']

In [5]:
def clean_text(sentences):
    result = []
    for sentence in sentences:
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        sentence = re.sub("\s+[a-zA-Z]\s+", ' ', sentence)
        sentence = re.sub('\s+', ' ', sentence)
        sentence = sentence.strip().lower()
        result.append(sentence)
    return result

In [6]:
def stop_and_stem(text):
#     ps = PorterStemmer() 
    lemmatizer = WordNetLemmatizer() 

    '''
    Removes stopwords and does stemming
    '''
    stoplist = stopwords.words('english')
#     stoplist.extend(common)
    
    text_stemmed = [lemmatizer.lemmatize(word) for word in word_tokenize(text) if word not in stoplist and len(word) > 3]
    text = ' '.join(text_stemmed)
    return text

# twentyone_data['Purpose'] = twentyone_data['Purpose'].apply(stop_and_stem)

In [7]:
a = 'Wells Fargo must/must not/may/should/could/can/cannot'
b = 'Account holder must/must not/may/should/could/can/cannot'
c = 'body text must/should'
d = 'bold text must/should'

In [8]:
# def get_text(x):
#     s = [i.lower() for i in x.split('\n') if i!='']
#     lst = []
#     for i in s:
#         if 'wells fargo must' in i or 'wells fargo must not' in i or 'wells fargo may' in i or 'wells fargo should' in i or 'wells fargo could' in i or 'wells fargo can' in i or 'wells fargo cannot' in i:
#             return i
#         elif 'account holder must' in i or 'account holder must not' in i or 'account holder may' in i or 'account holder should' in i or 'account holder could' in i or 'account holder can' in i or 'account holder cannot' in i:
#             return i
#         elif 'body text must' in i or 'body text should' in i:
#             return i
#         elif 'bold text must' in i or 'bold text should' in i:
#             return i

In [9]:
def get_text(x):
    s = [i.lower() for i in x.split('\n') if i!='']
    lst = []
    for i in s:
        if 'wells fargo' in i:
            return i
        elif 'account holder' in i:
            return i
#         elif 'body text must' in i or 'body text should' in i:
#             return i
#         elif 'bold text must' in i or 'bold text should' in i:
#             return i

In [10]:
def get_text_1(x):
    indx = x.find('1. ')
    
    if indx!=-1:
        s = x[:indx]
    else:
        s = x
        
    s = [i.lower() for i in s.split('\n') if i!='']
    lst = []
    for i in s:
        if 'wells fargo' in i:
            return i
        elif 'account holder' in i:
            return i
#         elif 'body text must' in i or 'body text should' in i:
#             return i
#         elif 'bold text must' in i or 'bold text should' in i:
#             return i

In [11]:
df

,Title,Summary
0,"PSDR - Contracts - Contract Terms, Notices, an...","Covered States: AK, AL, AR, AZ, CA, CO, CT, DC..."
1,"PSDR - Duty to notify of forgeries, alteration...","MCR 1100\n \n Covered States: AK, AL, AR, AZ, ..."
2,PSDR - Contracts - Transferability and Assignm...,Cluster: PSDR - Contracts - Transferability an...
3,"PSDR - Contracts - Contract Terms, Notices and...","Cluster: PSDR - Contracts - Contract Terms, No..."
4,PSDR - Contracts and Disclosures - Form and St...,Cluster: PSDR - Contracts and Disclosures - Fo...
...,...,...
97,PSDR - Disclosures - Prescribed Language - Ser...,"Covered States: CT, FL, IL, MI, MN, NY, OR, TX..."
98,PSDR - Retail Leases - Goods (including Motor ...,"Covered States: CA, CO, CT, DC, GA, LA, OK, PR..."
99,PSDR - Retail Leases - Secured Party's Contrac...,"Covered States: AK, AL, AZ, AR, CA, CO, CT, DC..."
100,"PSDR - Retail Sales - Leases - Exclusion, Modi...","Covered States: AK, AL, AR, AZ, CA, CO, DC, DE..."


In [12]:
df['text'] = df['Summary'].apply(lambda x:get_text_1(x))

In [13]:
df["text"] = df["text"].astype(str)

In [14]:
df["text"]

0       wells fargo must not allow a borrower or guar...
1       as a sending bank of a payment order, wells f...
2       when engaged in the transfer or assignment of...
3       when providing a loan or offering certain pro...
4       when offering the consumer contracts, product...
                             ...                        
97      when providing consumer credit or servicing c...
98      wells fargo must disclose all information for...
99      in a contract for the sale, lease, license, o...
100     when excluding or modifying a lease warranty ...
101     as a lender in certain loan transactions secu...
Name: text, Length: 102, dtype: object

In [15]:
df["text_clean"] = clean_text(df["text"].tolist())
df["text_clean"] = df["text_clean"].apply(stop_and_stem)

In [16]:
from collections import Counter
Counter(" ".join(filter(None, df["text_clean"])).split()).most_common(15)

[('well', 129),
 ('must', 129),
 ('fargo', 128),
 ('certain', 66),
 ('disclosure', 58),
 ('provide', 47),
 ('specific', 45),
 ('consumer', 45),
 ('loan', 44),
 ('state', 39),
 ('notice', 39),
 ('agreement', 35),
 ('product', 31),
 ('credit', 30),
 ('information', 29)]

In [17]:
x = df['text_clean'].apply(lambda x: [' '.join(ng) for ng in everygrams(word_tokenize(x), 2, 3)]).to_frame()

In [18]:
flattenList = pd.Series(np.concatenate(x.text_clean))
freqDf = flattenList.value_counts().sort_index().rename_axis('text').reset_index(name = 'frequency')

In [19]:
freqDf.sort_values('frequency', axis = 0, ascending = False, inplace = True, na_position= 'last')

In [20]:
# freqDf

In [21]:
# freqDf.to_csv(r'\Users\svatsavai001\Documents\bigramstrigrams_0407MCRs.csv')

In [22]:
# import matplotlib.pyplot as plt 

# plt.figure()
# flattenList.value_counts().plot(kind = 'bar', title = 'Count of 1-word and 2-word frequencies')
# plt.xlabel('Words')
# plt.ylabel('Count')
# plt.show()

In [23]:
def what(x):
    if x=='None':
        return False
    
    pattern = r'\bmust\b|\bmust not\b|\bshall\b|\bshall not\b|\bcould\b|\bcan\b|\bcannot\b|\bwill\b|\bwill not\b'
    found = re.findall(pattern, x, re.IGNORECASE)
    
    if found:
        return True
    else:
        return False

In [24]:
def who(x):
    
    if x=='None':
        return False
    
    pattern = r'\bwells fargo\b|\baccount holder\b|'
    found = re.findall(pattern, x, re.IGNORECASE)
    
    if found:
        return True
    else:
        return False

In [25]:
def when(x):
    
    if x=='None':
        return False
    
    pattern = r'\bwhen\b|\bfor\b|\bin\b|\bas\b|\bif\b'
    found = re.findall(pattern, x, re.IGNORECASE)
    
    if found:
        return True
    else:
        return False

In [26]:
def who_may(x):
    
    if x=='None':
        return True
    
    pattern = r'\bwells fargo may\b|\baccount holder may\b'
    found = re.findall(pattern, x, re.IGNORECASE)
    
    if found:
        return False
    else:
        return True

In [35]:
def capitalization (x):
    
    if x=='None':
        return None
    
    pattern = r'\bWells Fargo\b'
    found1 = re.findall(pattern, x)
    found2 = re.findall(pattern, x,re.IGNORECASE)
    
    
    if len(found1)==len(found2) and len(found1)>0:
        return True
    elif len(found2)>0:
        return False
    else:
        return None

In [36]:
df['What'] = df['text'].apply(lambda x:what(x))
df['When'] = df['text'].apply(lambda x:when(x))
df['Who'] = df['text'].apply(lambda x:who(x))

df['Clear_Decisive'] = df['Summary'].apply(lambda x:who_may(x))
df['capitalization'] = df['Summary'].apply(lambda x:capitalization(x))

In [37]:
df[df['Clear_Decisive']==True]

,Title,Summary,text,text_clean,What,When,Who,Clear_Decisive,capitalization
0,"PSDR - Contracts - Contract Terms, Notices, an...","Covered States: AK, AL, AR, AZ, CA, CO, CT, DC...",wells fargo must not allow a borrower or guar...,well fargo must allow borrower guarantor waive...,True,False,True,True,True
1,"PSDR - Duty to notify of forgeries, alteration...","MCR 1100\n \n Covered States: AK, AL, AR, AZ, ...","as a sending bank of a payment order, wells f...",sending bank payment order well fargo must exe...,True,True,True,True,True
2,PSDR - Contracts - Transferability and Assignm...,Cluster: PSDR - Contracts - Transferability an...,when engaged in the transfer or assignment of...,engaged transfer assignment account mortgage w...,True,True,True,True,True
3,"PSDR - Contracts - Contract Terms, Notices and...","Cluster: PSDR - Contracts - Contract Terms, No...",when providing a loan or offering certain pro...,providing loan offering certain product servic...,True,True,True,True,True
4,PSDR - Contracts and Disclosures - Form and St...,Cluster: PSDR - Contracts and Disclosures - Fo...,"when offering the consumer contracts, product...",offering consumer contract product service lis...,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...
96,PSDR - Retail Leases - Consumer Lease Provisio...,"Covered States: AK, AL, AR, AZ, CA, CO, CT, DC...","when acting as a lessor, wells fargo must use...",acting lessor well fargo must specific written...,True,True,True,True,True
97,PSDR - Disclosures - Prescribed Language - Ser...,"Covered States: CT, FL, IL, MI, MN, NY, OR, TX...",when providing consumer credit or servicing c...,providing consumer credit servicing consumer l...,True,True,True,True,True
99,PSDR - Retail Leases - Secured Party's Contrac...,"Covered States: AK, AL, AZ, AR, CA, CO, CT, DC...","in a contract for the sale, lease, license, o...",contract sale lease license disposition collat...,True,True,True,True,True
100,"PSDR - Retail Sales - Leases - Exclusion, Modi...","Covered States: AK, AL, AR, AZ, CA, CO, DC, DE...",when excluding or modifying a lease warranty ...,excluding modifying lease warranty interferenc...,True,True,True,True,True


In [38]:
df[df['text']=='None']

,Title,Summary,text,text_clean,What,When,Who,Clear_Decisive,capitalization
39,PSDR - Disclosures - Servicer Disclosures - Bo...,"Covered States: AR, CA, CO, DE, GA, HI, IN, LA...",None,none,False,False,False,False,True
52,"PSDR - Duty to notify of forgeries, alteration...",Nothing in this cluster appears to be actionab...,None,none,False,False,False,True,None
71,PSDR - Contracts and Disclosures - Form and St...,MCR1026\n \n Covered State: MI\n \n No general...,None,none,False,False,False,True,None
80,PSDR - Credit Application Notices - Fee Disclo...,"MCR1088\n \n Covered States: HI, IA, IL, LA, M...",None,none,False,False,False,True,True


In [39]:
# df.to_csv(r'\Users\svatsavai001\Documents\WhoWhatWhentest_0407_100MCRs_v3.csv')

In [34]:
df['Summary'][71]

'MCR1026\n \n Covered State: MI\n \n No generally applicable supporting citations.\n \n 1. Lending Product (loans, lines and leases) - Asset Based Loans;\n 2. Lending Product (loans, lines and leases) - Consumer Credit Cards;\n 3. Lending Product (loans, lines and leases) - Consumer Other Loans/Lines;\n 4. Lending Product (loans, lines and leases) - Consumer Real Estate;\n 5. Lending Product (loans, lines and leases) - Retail Broker Dealer - Products;\n 6. Lending Product (loans, lines and leases) - Structured Products; and\n \n State Specific Restrictive Requirements:\n \n Mich. Admin. Code ¬ß 445.1005'

In [33]:
df.iloc[76]

Title             PSDR - Credit Application Notices - ARM and Va...
Summary           Cluster: PSDR - Credit Application Notices -AR...
text               when offering variable rate loans, wells farg...
text_clean        offering variable rate loan well fargo must di...
What                                                           True
When                                                           True
Who                                                            True
Clear_Decisive                                                 True
capitalization                                                False
Name: 76, dtype: object